# Learn the basics of Blockchain with Julia
## Part 2 - Basic Blockchain simulation

So far we have explored hashing functions, private keys, signature verification, and other cryptographic functionalities. These are all important ideas that are not specific to blochchain yet play a pivotal role in the ecosystem. Next up, we will be looking at a really simple blockchain simulation in order to get a better idea of how we could build a blockchain in Julia.

In [1]:
# Code inspired by: https://gist.github.com/TestSubjector/efae35941009dd5dc33aa639ff439b3f#file-blockchain-jl
using SHA
using Dates

In [2]:
"""
An individual block structure
"""
struct Block
    index::Int
    timestamp::DateTime
    data::String
    previous_hash::String # length 64 string encoding of hexadecimal number
    hash::String
    nonce::Int
    
    # In a real blockchain block, we would want some previous transactions in here rather than just generic data.
    
    """
    The constructor which will also create hash for the block.
    It will use 256 bit encryption for the blockchain's security needs.
    """
    function Block(index, timestamp, data, previous_hash, nonce)
        hash = sha2_256(string(index, timestamp, data, previous_hash))
        nonce += 1
        new(index, timestamp, data, previous_hash, bytes2hex(hash), nonce)
    end
end

Block

In [3]:
"""
Add another block to the chain.
You can't have a blockchain with just one block after all.
"""
function addBlock(tail_block::Block)
    new_index = tail_block.index + 1
    return Block(new_index, Dates.now(), string("This is block", new_index), tail_block.hash, tail_block.nonce)
end

addBlock

In [4]:
# All blockchains have a "Genesis Block" or the block that kicks off that specific Blockchain.
Blockchain = [Block(0, Dates.now(), "Genesis Block", "0", 0)]

1-element Vector{Block}:
 Block(0, DateTime("2021-04-25T11:05:23.802"), "Genesis Block", "0", "9451c7fb9f23c22e29c003890f79b8779736b771f5b988bad938c3c488a7c42c", 1)

In [5]:
# Now we will configure the max size of the blockchain
Blockchain_limit = 20

20

In [6]:
function runBlockSimulation()
    for tail = 1:Blockchain_limit
        # Link the new block to the chain
        append!(Blockchain, [addBlock(Blockchain[tail])])

        # The details of the block
        println("Block : $(tail+1)")
        println("Hash :", Blockchain[tail+1].hash)
    end
end
runBlockSimulation()

Block : 2
Hash :1bbb565fee80ec13da5205ad478f38d93a34e36c965bd0d55e14645da92824cc
Block : 3
Hash :dae41ce7f2f8377efb5f809e1f612632a907f53f7b120f2068e5990b10754fb8
Block : 4
Hash :ff342c44cb49e6bf982909d6618da930c48844d1bdb2ee72c98510d5c5889cbe
Block : 5
Hash :a7a42a36d3cba3478b4b949c0bfa5768ec5bb834b39ce43a9b72938b68f69da3
Block : 6
Hash :1c9cc0f852ed2ab9852bee12cbbff6549a508d77a9f9e0efbc0f870acdbab5de
Block : 7
Hash :3507d9cd3c9be56fb68d6899ecdf871cb45392cf8c9b9fb670cbcfa4a9aa33bf
Block : 8
Hash :f3cc1fafecc4ad2ade661097d6ec930c208c14f2022842f189c1df63a3673e56
Block : 9
Hash :54d3af3e593816aac02442e08a32952dcfe04049d8b508015a1e2f3e62b163ef
Block : 10
Hash :8233cdcb4ae98f66ace7d0eb4353278cf081b21fb5587412db26e3765d22d72e
Block : 11
Hash :f4bc9145cf39a97e0346a806270b88603236d3d54fdae3726960f6bddd5804eb
Block : 12
Hash :a85ff779f58bfd72ac3b9f3f25a31bd8417f7250b5f24e59245a67985847a751
Block : 13
Hash :28bdea798740233f3f03c2b694d11ffb7172089636105f4c142ffa6b3960ebdc
Block : 14
Hash :b1d2a33

In [7]:
Blockchain[20]

Block(19, DateTime("2021-04-25T11:05:25.485"), "This is block19", "9195e90df70e065410dbd7660d13f6c36ff9bb05f172273018ef412f6abd9163", "b5d967348628e2e6cf332bc4e1eb6bba5b721c0e9c77083dd6cf142e3f815b13", 20)

Let's note a few things quickly, first, this blockchain is not setup with any notion of transactions, rewards, etc. which are all key ideas in the context of blockchain development. Part of the reason for this is that we have no setup currently to handle accounts or the like. This would be required for us to have a real blockchain but we will graze over this fact for now and look at how we can get this example closer to where we want it starting by adding a transaction field to the Block defintion. 

In [34]:
# We would techncially want to explicitly type this to only work with arrays of newBlocks
function hashTransactions(transactions)
    
    transactions_hash_str = ""
    
    # Concatenate all the hash values together
    for transaction in transactions
        transactions_hash_str *= transaction.hash
    end
    print("Transactions concatenated")
    
    # Return the Concatenated hash values
    return(bytes2hex(sha2_256(transactions_hash_str)))
end

hashTransactions (generic function with 2 methods)

In [9]:
"""
An individual block structure
"""
mutable struct newBlock
    index::Int
    timestamp::DateTime
    data::String
    previous_hash::String # length 64 string encoding of hexadecimal number
    hash::String
    nonce::Int
    transactions::Array{newBlock} # Can be none or contain N blocks
    transactions_hash::String
    
    # In a real blockchain block, we would want some previous transactions in here rather than just generic data.
    
    """
    The constructor which will also create hash for the block.
    It will use 256 bit encryption for the blockchain's security needs.
    """
    function newBlock(index, timestamp, data, previous_hash, nonce, transactions = [])
        hash = sha2_256(string(index, timestamp, data, previous_hash))
        nonce += 1
        
        transactions_hash = ""
        if transactions != []
            transactions_hash = hashTransactions(transactions)
        end
        new(index, timestamp, data, previous_hash, bytes2hex(hash), nonce, transactions, transactions_hash)
    end
end

newBlock

In [10]:
"""
Add another block to the chain.
You can't have a blockchain with just one block after all.
"""
function addNewBlock(tail_block::newBlock)
    new_index = tail_block.index + 1
    return newBlock(new_index, Dates.now(), string("This is new block", new_index), tail_block.hash, tail_block.nonce)
end

addNewBlock

In [11]:
newBlockchain = [newBlock(0, Dates.now(), "New Genesis Block", "0", 0)]

1-element Vector{newBlock}:
 newBlock(0, DateTime("2021-04-25T11:05:26.115"), "New Genesis Block", "0", "b3a7c99e9d33c2099f8b1b26fff377f6cdb53155b96f8c968d2bc3905ccec073", 1, newBlock[], "")

In [12]:
function runNewBlockSimulation()
    for tail = 1:Blockchain_limit
        # Link the new block to the chain
        append!(newBlockchain, [addNewBlock(newBlockchain[tail])])

        # The details of the new block
        println("Block : $(tail+1)")
        println("Hash :", newBlockchain[tail+1].hash)
    end
end
runNewBlockSimulation()

Block : 2
Hash :0f1abcec75c0da6786d92ef526014b6fa336891e6464a09511e4330d7c70a5c7
Block : 3
Hash :01cd8e708f49c985178b2ec9d7b33656b18615d67b285133c4ecffd09fa09178
Block : 4
Hash :230459e174ceacb22e91acbfe779c6a8b0e772f39743b503ff99558c4d21e6c6
Block : 5
Hash :ad673797192e3eef3317facf2890892bddafb1d823efa76f533ef512303ab23a
Block : 6
Hash :5e18d5719a3967ff73895ea95294f516419cdb6827b2a183b02b2a7748a8e7d7
Block : 7
Hash :05cee313b671fa9a433f34a49c72b4892b1ee7afc0e62b945998e29cb552f19b
Block : 8
Hash :d99c19a525aa294e4d72032c100e7744e7ac8691ae8c40e0e7beb14a6e696a01
Block : 9
Hash :4044bae3ca0802ee04d78712cb7111174add2b1814bb201ebd8c3436d6155140
Block : 10
Hash :6d25a1882b5e25440bcbc03710d77d8bcdfccbdc23aba4bd9bf37da365e83a3d
Block : 11
Hash :e4029eae6a421b5a96d7a057b1312abcceefe67a40eaffe69a61142954a7ab00
Block : 12
Hash :ee25228a44d239c2763c75f7c5b91a6bd7926d2f55cf804cde622ec1c8a8eacf
Block : 13
Hash :63bfd050fabd98b2c84039f2ec5cdd782082a7b419468bb98ff5f5cc58a63987
Block : 14
Hash :d03a64a

In [14]:
newBlockchain[21].transactions = [newBlockchain[20]]

1-element Vector{newBlock}:
 newBlock(19, DateTime("2021-04-25T11:05:26.925"), "This is new block19", "a7b552cdedaf8f0b9f133223f8c5fb0bca8fdce422eee246525141dc046d407d", "2a4e94b35603f2060fd7e8d58a9f9e9749052982f447e63029b4dc10a42e232a", 20, newBlock[], "")

In [16]:
newBlockchain[21].transactions_hash

""

On first glance, it looks like something here is off given we have transactions in the block but no transaction hash. This is a built in safety mechanism to ensure that transcations are not added to the block after the fact. The block requires that the transactions are passed in when it is created so that a hash can be generated. Let's see what that looks like below...

In [35]:
testBlock = newBlock(0, Dates.now(), "A Random Block", "0", 0, [newBlockchain[10]])

Transactions concatenated

newBlock(0, DateTime("2021-04-25T11:25:55.476"), "A Random Block", "0", "6cc6787fbcfe9bd9361dc8a2b1bfe1eb6664a5db0c4de73ca8ed6d7340ca14e4", 1, newBlock[newBlock(9, DateTime("2021-04-25T11:05:26.924"), "This is new block9", "4044bae3ca0802ee04d78712cb7111174add2b1814bb201ebd8c3436d6155140", "6d25a1882b5e25440bcbc03710d77d8bcdfccbdc23aba4bd9bf37da365e83a3d", 10, newBlock[], "")], "6e902e927414707275a7ced026a05dcc13ecde029861594d95c23e3b8744351d")

We now have the idea of transactions being encapsulted into the block itself. 

In [36]:
testBlock = newBlock(0, Dates.now(), "A Random Block", "0", 0, [newBlockchain[10], newBlockchain[11]])

Transactions concatenated

newBlock(0, DateTime("2021-04-25T11:25:58.605"), "A Random Block", "0", "9e541d9cef08c3b8c93ae910bab15d3095d28b66629a226e4019bdf7f8d1c762", 1, newBlock[newBlock(9, DateTime("2021-04-25T11:05:26.924"), "This is new block9", "4044bae3ca0802ee04d78712cb7111174add2b1814bb201ebd8c3436d6155140", "6d25a1882b5e25440bcbc03710d77d8bcdfccbdc23aba4bd9bf37da365e83a3d", 10, newBlock[], ""), newBlock(10, DateTime("2021-04-25T11:05:26.924"), "This is new block10", "6d25a1882b5e25440bcbc03710d77d8bcdfccbdc23aba4bd9bf37da365e83a3d", "e4029eae6a421b5a96d7a057b1312abcceefe67a40eaffe69a61142954a7ab00", 11, newBlock[], "")], "57015387487f0df84112715f0af890d4689711802a3f251aa6546bcd05b4ff6f")

In [38]:
testBlock.transactions_hash

"57015387487f0df84112715f0af890d4689711802a3f251aa6546bcd05b4ff6f"

While not fully functional by any means, this simulation gives us a glipse into what would be required for a functional blockchain to work in Julia. Some key points missing: a fully functional transaction (up to this point we just used basic blocks as transactions despite that not being correct), account registry (should be easy enough to create in Julia), and more. The real question mark still at this point related to all of this is generating public keys. This is a critial part of the account registry and something that would need to be implimented.